In [9]:
import os
os.environ['SSL_CERT_FILE'] = '/path/to/certfile'
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


In [30]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import MaxPool2D, Conv2D, Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical # one-hot encoding : 크로스 엔트로피 값을 계산하기 위해 원핫인코딩 해줘야함.
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

(X_train, y_train), (X_test,y_test) = mnist.load_data()

X_train = X_train.reshape(60000,28,28,1) / 255
X_test = X_test.reshape(10000,28,28,1) / 255
# X_traon.shape => (60000,  28, 28) => 뒤에 1자리 하나 만들어줘야함. => .reshape 해주기

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

* tf.keras.layers.Conv2D(
    filters,
    kernel_size,
    strides=(1, 1),
    padding='valid',
    data_format=None,
    dilation_rate=(1, 1),
    groups=1,
    activation=None,
    use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
    **kwargs
)
* tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2),
    strides=None,
    padding='valid',
    data_format=None,
    **kwargs
)
* tf.keras.layers.Dropout(
    rate, noise_shape=None, seed=None, **kwargs
)

* tf.keras.layers.Flatten(
    data_format=None, **kwargs
)



In [31]:
# CNN 구조

model = Sequential()
model.add(Conv2D(filters=32, kernel_size = (3,3), padding="same", activation = 'relu'))
model.add(Conv2D(filters=64, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(10,activation = 'softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

너무 오래 걸리고 발열 심해서 안돌림 -> 구글 코랩으로 연동해서 돌리는 걸 추천 TPU로 돌리기.
결과는 교수님꺼 보기.

In [33]:
# 모델 최적화 
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath = "./model/{epoch:02d}-{val_loss:4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, moniter = 'val_loss', verbose=1, save_best_only = True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

# 모델의 실행
history = model.fit(X_train, y_train, validation_data=(X_test,y_test),\
                    epochs = 30, batch_size = 200, verbose=0, callbacks=[early_stopping_callback, checkpointer])

# 테스트 정확도 출력
print("\n Test Accuracy : %.4f" % (model.evaluate(X_test, y_test)[1]))

# 테스트 셋의 오차
y_vloss = history.history['val_loss']

# 학습 셋의 오차
y_loss = history.history['loss']

# 그래프로 표현
x_len = numpy.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker = '.', c = 'red', label = 'Testset_loss')
plt.plot(x_len, y_loss, marker = '.', c = 'blue', label = 'Trainset_loss')

# 그래프에 그리드를 주고 레이블을 표시
plt.legend(loc = 'upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()


Epoch 1: val_loss improved from inf to 0.05168, saving model to ./model/01-0.051681.hdf5


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 0.05168 to 0.03931, saving model to ./model/02-0.039306.hdf5


KeyboardInterrupt: 